In [1]:
import numpy as np
import multiprocessing
import tqdm

import sys

import utils
from data import _hic_dataset_pyg

In [3]:
# we define the parameters for the preprocessing step.
nProc = 24

nMin_node = (
    31 + 1
)  # target graph will have min of 32 nodes so that we can remove 1 node and still have 31 nodes for query graph
nMax_node = 51
use_prcnt = 20

npz_path = "matrix/HeLa_10000.obs_exp_qt.npz"
samples = ["4DNFIBM9QCFG"]  # Samples to include in the training dataset

In [4]:
# Get the keys for the npz file that contains the samples for training
npz = np.load(npz_path)
npz_keys = [k for k in npz.files if k.split("-")[0] in samples]

In [5]:
# Construct the training dataset for the GNN model, in which the synthetic graphs will be sampled.
args = tqdm.tqdm(
    [(npz[k], nMin_node, nMax_node, None, use_prcnt, None) for k in npz_keys]
)

with multiprocessing.Pool(processes=nProc) as pool:
    results = pool.starmap(utils.constGraphList, args)

# 23 * 2 = 46 chromosome arms

100%|██████████| 46/46 [05:55<00:00,  7.72s/it]


In [6]:
# pack constructed graphs into a data_list
# tup is a tuple of (graph_list, corresponding matrix indexes)
data_list = [g for tup in results for g in tup[0]]

In [7]:
# Create an InMemoryDataset for training from the graphs in the data_list, an optimal way to save graph datasets
root = f"matrix/hic_{'_'.join([s for s in samples])}_nMax{nMax_node}_nMin{nMin_node}_perc{use_prcnt}_log10"
_hic_dataset_pyg(root=root, data_list=data_list)

Processing...
Done!


_hic_dataset_pyg(48162)